In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet152

In [2]:
x_train = np.load('x_train.npy')
y_train = np.load('y_train.npy')

x_validation = np.load('x_validation.npy')
y_validation = np.load('y_validation.npy')

x_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')

In [3]:
##### CNN model creation

In [4]:
resnet152 = ResNet152(include_top=False, weights='imagenet', input_shape=x_train.shape[1:])

In [5]:
model = Sequential([
  resnet152
])


In [6]:
model.add(GlobalAveragePooling2D()) # test avec flatten?
model.add(Dropout(0.2))

model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.2))

#model.add(Dense(128, activation = 'relu'))
#model.add(Dropout(0.2))

model.add(Dense(7, activation = 'softmax'))
#model.add(Activation('softmax'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152 (Model)            (None, 6, 8, 2048)        58370944  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 3591      
Total params: 59,423,623
Trainable params: 59,272,199
Non-trainable params: 151,424
______________________________________

In [8]:
model.load_weights('resnet152_weight')

In [8]:
########################## Common part: optimization and training#############################################

In [32]:
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [33]:
model.compile(optimizer= optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [34]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.000001)

In [35]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images
datagen.fit(x_train)

In [36]:
class_weights = {0: 10015/327/30,
                1: 10015/514/30,
                2: 10015/1099/30,
                3: 10015/115/30,
                4: 10015/1113/30,
                5: 10015/6705/30,
                6: 10015/142/30}

In [37]:
class_weights

{0: 1.0208970438328238,
 1: 0.6494811932555123,
 2: 0.30376099484379737,
 3: 2.9028985507246374,
 4: 0.29994010182689423,
 5: 0.049788714889386035,
 6: 2.3509389671361505}

In [38]:
epochs = 5 
batch_size = 10
history = model.fit_generator(
    datagen.flow(x_train,y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs,
    validation_data=(x_validation,y_validation),
    validation_steps=x_validation.shape[0] // batch_size,
    callbacks=[learning_rate_reduction],
    class_weight = class_weights
)

Epoch 1/5
630/630 [==============================] - 853s 1s/step - loss: 0.2984 - accuracy: 0.5112 - val_loss: 1.4260 - val_accuracy: 0.4180
Epoch 2/5
630/630 [==============================] - 848s 1s/step - loss: 0.3134 - accuracy: 0.5118 - val_loss: 1.3879 - val_accuracy: 0.4907
Epoch 3/5
630/630 [==============================] - 844s 1s/step - loss: 0.3026 - accuracy: 0.5233 - val_loss: 1.6076 - val_accuracy: 0.4494
Epoch 4/5
630/630 [==============================] - 840s 1s/step - loss: 0.3008 - accuracy: 0.5158 - val_loss: 1.3626 - val_accuracy: 0.5064
Epoch 5/5
630/630 [==============================] - 834s 1s/step - loss: 0.2998 - accuracy: 0.5029 - val_loss: 1.4259 - val_accuracy: 0.3866


In [31]:
#### Saving the weights of the model for a later use.


model.save_weights('resnet152_weight')

In [27]:
loss, accuracy = model.evaluate(x_test, y_test, verbose = 1)

3005/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [28]:
##### confusion matrix

pred = model.predict(x_test)
pred_label = np.zeros(len(y_test), dtype='uint8')

for i in range(len(pred)):
    pred_label[i] = int(np.argmax(pred[i]))
    

In [29]:
confusion = confusion_matrix(y_test, pred_label)

In [30]:
print(confusion)
print('predicted is horizontal, true label is vertical')

[[  53    2   16   17    3    0    3]
 [  58    7    8   66    2    1    4]
 [  79    3  166   26   70    5    5]
 [  12    0    1   11    1    1    1]
 [  30    2   95    2  169   13    8]
 [ 112    6  164   86  486 1116   47]
 [   0    1    0    1    3    0   43]]
predicted is horizontal, true label is vertical
